### 1. Import library

In [2]:
import emoji
from underthesea import word_tokenize, sent_tokenize
from viet_text_tools import normalize_diacritics
import re 
import pandas as pd

import string
import os

### 2. Data Loading 

In [33]:
file_names = ["banphim_comments.csv", "banvedientu_comments.csv", "chuot_comments.csv", "danamthanh_comments.csv", "dienthoai_phukien_comments.csv", "gameconsole_comments.csv",
              "laptop_comments.csv", "linhkienmaytinh_comments.csv", "pc_comments.csv", "maynghenhac_comments.csv", "micro_comments.csv", "tainghecoday_comments.csv",
              "tivi_comments.csv", "sac_comments.csv", "maytinhbang_comments.csv","machdientu_comments.csv","tainghekhongday_comments.csv"]

dir = 'E:\MY CAREER\Internship Project\EDA\dataset\shopee\comment'
dataframes = []
for file in file_names:
    path = os.path.join(dir, file)
    try:
        df = pd.read_csv(path)
        dataframes.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file")

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head()


,Unnamed: 0,orderId,itemId,shopId,cmtId,rating,comment,ship_oversea,region,userId,likes
0,0,122029931276914,1546899006,93922606,10092536990,5,"Sản phẩm đẹp, đóng gói cẩn thận, tuy chưa kết ...",False,vn,792910715,25.0
1,1,126678434282902,1546899006,93922606,10515236390,5,Chất lượng sản phẩm:tốt\nTính năng nổi bật:bàn...,False,vn,36121190,32.0
2,2,117545382204991,1546899006,93922606,9644909661,5,Chất lượng sản phẩm:cũng được\nTính năng nổi b...,False,vn,729345506,41.0
3,3,130741239293049,1546899006,93922606,10851288939,5,Tính năng nổi bật:có đèn siêu đẹp\nChất lượng ...,False,vn,343048455,8.0
4,4,130771099281440,1546899006,93922606,10875021785,5,Chất lượng sản phẩm:tốt\nTính năng nổi bật:khô...,False,vn,748343507,4.0


In [34]:
combined_df.columns

Index(['Unnamed: 0', 'orderId', 'itemId', 'shopId', 'cmtId', 'rating',
       'comment', 'ship_oversea', 'region', 'userId', 'likes'],
      dtype='object')

In [35]:
combine = combined_df[['orderId', 'itemId', 'shopId', 'cmtId', 'rating',
       'comment', 'ship_oversea', 'region', 'userId', 'likes']]
combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234379 entries, 0 to 234378
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   orderId       234379 non-null  int64  
 1   itemId        234379 non-null  int64  
 2   shopId        234379 non-null  int64  
 3   cmtId         234379 non-null  int64  
 4   rating        234379 non-null  int64  
 5   comment       168903 non-null  object 
 6   ship_oversea  234379 non-null  bool   
 7   region        234379 non-null  object 
 8   userId        234379 non-null  int64  
 9   likes         53499 non-null   float64
dtypes: bool(1), float64(1), int64(6), object(2)
memory usage: 16.3+ MB


### 3. Data Cleaning

Xóa các dòng dữ liệu không có comment

In [36]:
combine = combine.dropna(subset='comment')
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168903 entries, 0 to 234369
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   orderId       168903 non-null  int64  
 1   itemId        168903 non-null  int64  
 2   shopId        168903 non-null  int64  
 3   cmtId         168903 non-null  int64  
 4   rating        168903 non-null  int64  
 5   comment       168903 non-null  object 
 6   ship_oversea  168903 non-null  bool   
 7   region        168903 non-null  object 
 8   userId        168903 non-null  int64  
 9   likes         50898 non-null   float64
dtypes: bool(1), float64(1), int64(6), object(2)
memory usage: 13.0+ MB


Xóa các comment bị trùng lặp

In [37]:
combine.duplicated(subset='cmtId').sum()

13030

In [38]:
combine.drop_duplicates(subset='cmtId', inplace=True, keep='last')

In [39]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155873 entries, 390 to 234369
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   orderId       155873 non-null  int64  
 1   itemId        155873 non-null  int64  
 2   shopId        155873 non-null  int64  
 3   cmtId         155873 non-null  int64  
 4   rating        155873 non-null  int64  
 5   comment       155873 non-null  object 
 6   ship_oversea  155873 non-null  bool   
 7   region        155873 non-null  object 
 8   userId        155873 non-null  int64  
 9   likes         47864 non-null   float64
dtypes: bool(1), float64(1), int64(6), object(2)
memory usage: 12.0+ MB


Kiểm tra các comment có emoji, nếu comment có emoji thì xóa 

In [93]:
filtered_data = combine
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F004-\U0001F0CF"  # CJK Compatibility Ideographs
                           u"\U0001F170-\U0001F251"  # Enclosed Ideographic Supplement
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

filtered_data["comment"] = filtered_data["comment"].apply(remove_emoji)

filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155873 entries, 390 to 234369
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   orderId       155873 non-null  int64  
 1   itemId        155873 non-null  int64  
 2   shopId        155873 non-null  int64  
 3   cmtId         155873 non-null  int64  
 4   rating        155873 non-null  int64  
 5   comment       155873 non-null  object 
 6   ship_oversea  155873 non-null  bool   
 7   region        155873 non-null  object 
 8   userId        155873 non-null  int64  
 9   likes         47864 non-null   float64
 10  word_count    155873 non-null  int64  
dtypes: bool(1), float64(1), int64(7), object(2)
memory usage: 13.2+ MB


Xóa các comment có teencode

In [94]:
with open('teencode.txt', "r", encoding="utf-8") as file:
    teencode_words = [line.strip() for line in file]

def get_lower(text):
    return text.lower().strip()

def remove_teencode_words(text):
    words = text.split()
    cleaned_words = [word for word in words if word not in teencode_words]
    return ' '.join(cleaned_words)

filtered_data['comment'] = filtered_data['comment'].apply(get_lower)
filtered_data['comment'] = filtered_data['comment'].apply(remove_teencode_words)
filtered_data["word_count"] = filtered_data["comment"].apply(lambda x: len(x.split()))

# Filter comments based on word count
filtered_data = filtered_data[(filtered_data["word_count"] > 5) & (filtered_data["word_count"] < 50)]

filtered_data

,orderId,itemId,shopId,cmtId,rating,comment,ship_oversea,region,userId,likes,word_count
391,115547853262706,3066084859,45654584,9473157284,5,"bàn phím nhạy, phím bấm êm, đèn led lên màu đẹ...",False,vn,844212025,8.0,27
393,110991500100515,3066084859,45654584,8984854294,5,"hàng đẹp xịn sò giống hình, chuột cầm nhẹ ta...",False,vn,636299992,2.0,34
395,89659632262758,3066084859,45654584,6522412723,5,bàn phím và chuột nhấn rất nhẹ nhàng. cảm ơn s...,False,vn,480477564,7.0,12
396,83580099086876,3066084859,45654584,5825350201,5,"sản phẩm được đóng gói kỹ càng, giao hàng nhan...",False,vn,435954539,2.0,19
397,87383887377285,3066084859,45654584,6358299273,5,"hàng tốt rất đẹp , hợp túi tiền , đóng gói rât...",False,vn,481142917,0.0,17
...,...,...,...,...,...,...,...,...,...,...,...
234365,132412034223284,15090822609,69687900,11039210979,5,shop phục vụ nhiệt tình chu đáo xử lý đơn hàng...,False,vn,252157545,NaN,17
234366,117980255227608,15090822609,69687900,9757454532,5,"giao hàng nhanh, đóng gói chỉn chu, cẩn thận, ...",False,vn,440977936,NaN,12
234367,141364883246851,15090822609,69687900,12034166975,5,"chất lượng sản phẩm:ok đeo hơi đau tai, gamemo...",False,vn,437423908,NaN,18
234368,116948237227127,15090822609,69687900,9619993014,5,sản phẩm này rất tốt cảm ơn shop rất nhiều cho...,False,vn,731941560,NaN,22


In [95]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130502 entries, 391 to 234369
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   orderId       130502 non-null  int64  
 1   itemId        130502 non-null  int64  
 2   shopId        130502 non-null  int64  
 3   cmtId         130502 non-null  int64  
 4   rating        130502 non-null  int64  
 5   comment       130502 non-null  object 
 6   ship_oversea  130502 non-null  bool   
 7   region        130502 non-null  object 
 8   userId        130502 non-null  int64  
 9   likes         40406 non-null   float64
 10  word_count    130502 non-null  int64  
dtypes: bool(1), float64(1), int64(7), object(2)
memory usage: 11.1+ MB


In [96]:
# Xóa badwords

with open('badwords.txt', "r", encoding="utf-8") as file:
    badwords = [line.strip() for line in file]

def remove_badwords(text):
    words = text.split()
    cleaned_words = [word for word in words if word not in badwords]
    return ' '.join(cleaned_words)

filtered_data['comment'] = filtered_data['comment'].apply(remove_badwords)


C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\2504681578.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(remove_badwords)


In [97]:
# Fill na like : 0
filtered_data['likes'] = filtered_data['likes'].fillna(0)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\714620834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['likes'] = filtered_data['likes'].fillna(0)


In [98]:
likes_count = len(filtered_data[filtered_data['likes'] == 0])
likes_count

95286

### 4. Data Exploring

In [99]:
def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')
    
    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )



In [100]:
filtered_data['comment'] = filtered_data['comment'].apply(convert_unicode)

filtered_data.head()

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\1194892130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(convert_unicode)


,orderId,itemId,shopId,cmtId,rating,comment,ship_oversea,region,userId,likes,word_count
391,115547853262706,3066084859,45654584,9473157284,5,"bàn phím nhạy, phím bấm êm, đèn led lên màu đẹ...",False,vn,844212025,8.0,27
393,110991500100515,3066084859,45654584,8984854294,5,"hàng đẹp xịn sò giống hình, chuột cầm nhẹ tay ...",False,vn,636299992,2.0,34
395,89659632262758,3066084859,45654584,6522412723,5,bàn phím và chuột nhấn rất nhẹ nhàng. cảm ơn s...,False,vn,480477564,7.0,12
396,83580099086876,3066084859,45654584,5825350201,5,"sản phẩm được đóng gói kỹ càng, giao hàng nhan...",False,vn,435954539,2.0,19
397,87383887377285,3066084859,45654584,6358299273,5,"hàng tốt rất đẹp , hợp túi tiền , đóng gói rât...",False,vn,481142917,0.0,17


In [101]:
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)


def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True


def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)


def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = word.split('/')
        cw = [standardize_word_typing(w) for w in cw]

        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_punctuation(text):
    punctuation = '!"#$%&\'()*+-/:;<=>?@[\\]^`{|}~'
    translator = str.maketrans(punctuation,' ' * len(punctuation))
    text = text.translate(translator)
    return text
    

In [102]:
def preprocess(text):
    text = text.strip()
    text = remove_punctuation(text)
    text = standardize_sentence_typing(text)
    return text 

filtered_data['comment'] = filtered_data['comment'].apply(preprocess)
filtered_data.head(5)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\3620772978.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(preprocess)


,orderId,itemId,shopId,cmtId,rating,comment,ship_oversea,region,userId,likes,word_count
391,115547853262706,3066084859,45654584,9473157284,5,"bàn phím nhaỵ, phím bấm êm, đèn led lên màu đẹ...",False,vn,844212025,8.0,27
393,110991500100515,3066084859,45654584,8984854294,5,"hàng đẹp xịn sò giống hình, chuột cầm nhẹ tay ...",False,vn,636299992,2.0,34
395,89659632262758,3066084859,45654584,6522412723,5,bàn phím và chuột nhấn rất nhẹ nhàng. cảm ơn s...,False,vn,480477564,7.0,12
396,83580099086876,3066084859,45654584,5825350201,5,"sản phẩm được đóng gói kỹ càng, giao hàng nhan...",False,vn,435954539,2.0,19
397,87383887377285,3066084859,45654584,6358299273,5,"hàng tốt rất đẹp , hợp túi tiền , đóng gói rât...",False,vn,481142917,0.0,17


In [103]:
def has_numbers(text):
    return bool(re.search(r'\d', text))

def remove_extra_dots(text):
    return re.sub(r'\.{2,}', '.', text)

with open('stopwords.txt', 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()

def remove_stopword(text):
    words = text.split()
    new_words = [word for word in words if word not in stopwords]
    return ' '.join(new_words)


def tokenize(text):
    return re.sub(r'\._+',' . ',word_tokenize(text, format="text"))

def start_processing(text):
    text = remove_extra_dots(text)
    text = tokenize(text)
    text = remove_stopword(text)



In [104]:
filtered_data['comment'] = filtered_data['comment'].apply(remove_extra_dots)
filtered_data.head(3)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\1003812478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(remove_extra_dots)


,orderId,itemId,shopId,cmtId,rating,comment,ship_oversea,region,userId,likes,word_count
391,115547853262706,3066084859,45654584,9473157284,5,"bàn phím nhaỵ, phím bấm êm, đèn led lên màu đẹ...",False,vn,844212025,8.0,27
393,110991500100515,3066084859,45654584,8984854294,5,"hàng đẹp xịn sò giống hình, chuột cầm nhẹ tay ...",False,vn,636299992,2.0,34
395,89659632262758,3066084859,45654584,6522412723,5,bàn phím và chuột nhấn rất nhẹ nhàng. cảm ơn s...,False,vn,480477564,7.0,12


In [105]:
filtered_data['comment'] = filtered_data['comment'].apply(tokenize)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\3662187122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(tokenize)


In [106]:
filtered_data['comment'] = filtered_data['comment'].apply(remove_stopword)

C:\Users\Rauciss\AppData\Local\Temp\ipykernel_7384\4249570693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['comment'] = filtered_data['comment'].apply(remove_stopword)


In [107]:
result = filtered_data[~filtered_data['comment'].str.contains(r'\d')]

In [108]:
result.columns

Index(['orderId', 'itemId', 'shopId', 'cmtId', 'rating', 'comment',
       'ship_oversea', 'region', 'userId', 'likes', 'word_count'],
      dtype='object')

In [109]:
result_df = result[['orderId', 'itemId', 'shopId', 'cmtId', 'rating', 'comment',
       'ship_oversea', 'region', 'userId', 'likes']]

result_df.to_csv('cleandata/shopee_comment.csv')

In [60]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14581 entries, 393 to 234369
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   orderId       14581 non-null  int64 
 1   itemId        14581 non-null  int64 
 2   shopId        14581 non-null  int64 
 3   cmtId         14581 non-null  int64 
 4   rating        14581 non-null  int64 
 5   comment       14581 non-null  object
 6   ship_oversea  14581 non-null  bool  
 7   region        14581 non-null  object
 8   userId        14581 non-null  int64 
 9   likes         14581 non-null  int32 
dtypes: bool(1), int32(1), int64(6), object(2)
memory usage: 1.1+ MB


In [112]:
result = pd.read_csv('cleandata/shopee_comment.csv')

In [113]:
def check_emoji(text):
    text_without_emoji = emoji.demojize(text)
    return text != text_without_emoji

result["has_emoji"] = result["comment"].apply(check_emoji)
result = result[result['has_emoji'] == False]

In [114]:
result['comment'] = result['comment'].apply(remove_stopword)
result['comment'] = result['comment'].apply(remove_extra_dots)
result['comment'] = result['comment'].apply(remove_teencode_words)



In [116]:
result.to_csv('cleandata/shopee_comment.csv')

In [3]:
full = pd.read_csv('cleandata/100k_shopee.csv')

In [5]:
total_rows = len(full)

# Tính số lượng dòng cho mỗi phần
part_size = total_rows // 6

# Chia DataFrame thành 6 phần
parts = [full[i:i + part_size] for i in range(0, total_rows, part_size)]

# Lưu mỗi phần thành một file CSV riêng
for i, part in enumerate(parts):
    part.to_csv(f'part_{i + 1}.csv', index=False)